#Convergence

$$
\newcommand{\DdQq}[2]{{\mathrm D}_{#1}{\mathrm Q}_{#2}}
\newcommand{\drondt}{\partial_t}
\newcommand{\drondx}{\partial_x}
\newcommand{\drondtt}{\partial_{tt}}
\newcommand{\drondxx}{\partial_{xx}}
\newcommand{\dx}{\Delta x}
\newcommand{\dt}{\Delta t}
\newcommand{\grandO}{{\mathcal O}}
\newcommand{\density}[2]{\,f_{#1}^{#2}}
\newcommand{\fk}[1]{\density{#1}{\vphantom{\star}}}
\newcommand{\fks}[1]{\density{#1}{\star}}
\newcommand{\moment}[2]{\,m_{#1}^{#2}}
\newcommand{\mk}[1]{\moment{#1}{\vphantom{\star}}}
\newcommand{\mke}[1]{\moment{#1}{e}}
\newcommand{\mks}[1]{\moment{#1}{\star}}
\newcommand{\norm}[1]{||#1||}
$$

In this work session, we investigate the convergence of the simple scheme $\DdQq{1}{2}$ for the advection equation in dimension one. The same analysis could be done for other schemes or other dimensions. 

##The $\DdQq{1}{2}$ for advection

We consider the problem
$$\drondt u + c\drondx u = 0,\qquad t>0, \qquad x\in(0,1),$$
with periodic boundary conditions (in order to not perturb the convergence rate). 
The simulation is performed with the following scheme
* the scheme velocity $\lambda=1$,
* the two velocities $v_0=1$, $v_1=-1$,
* the polynomials $P_0=1$, $P_1=X$ that defines the moments,
* the first moment $u$ is conserved and the second moment $v$ that relaxes toward its equilibrium value $cu$,
* the relaxation parameter $s\in[0,2]$.

####Question 1

Propose a dictionary for this $\DdQq{1}{2}$. 

####Question 2

Test the dictionary on a very simple test case: $s=1.$, $c=0.5$, $\dx=0.1$, and $$u(x,t=0) = \left\{ \begin{aligned} 1&\quad \text{if } 0<x<0.1, \\ 0 &\quad \text{else.} \end{aligned} \right. $$
In particular, plot the solution at several time.

##Convergence rate for smooth solutions

The Taylor analysis of the scheme (order of consistancy) can be justify for a smooth solution. We then compute the numerical convergence rate with a smooth initial function.

####Question 3

Propose a function `solution(x, t)` that is a smooth solution of the advection equation. It could be build with piecewise polynomial functions. 

####Question 4

Compute the numerical solution given by the $\DdQq{1}{2}$ and compare it to the exact solution.

####Question 5

Compute the error ($L^1$-norm) for several space steps (the space step could be $\dx=2^{-k}$ for k integer). Evaluate the order of the convergence of this scheme.

##Convergence rate for discontinuous solutions

####Question 6

Compute the order of convergence for a discontinuous solution by changing the initialization.

In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./custom.css", "r").read()
    return HTML(styles)
css_styling()